# Relational Algebra

Relational Algebra forms the basis for relational databases and the SQL query language.  In many ways understanding relational algebra is better than learning SQL right away as it is very consistent and extensible.  It is consitent because every relational algebra operator takes one or more relations as input and produces a relation as output.

Using the relational operators we can answer many questions.  In fact once you have mastered the relational operators it is almost trivial to translate a series of relational operators into SQL for use with the database of your choice.  As is the case with many aspects of computer science, any one of these relational operators is easy to understand in isolation, it is the combination of the operators that allows us to answer very complex questions.


## A Relation

A relation is one of the fundamental units of data used in Relational Algebra.  A relation is a collection of **tuples** where each tuple is composed of data.  The data in these tuples is in a consistent order, and would typically represent a collection of information about a particular entity.   To put it in terms you are probably more familiar with, a relation is like a spreadsheet, that consists of rows and columns of data.  The entire spreadsheet corresponds to the relation, and a row of the spreadsheet corresponds to a tuple.  In this notebook we will use two simple relations, a City relation and a Country relation.

Relations also have one more important property, which is that they do not have duplicate rows.

In the example below we will load two relations from files on disk, and display the first few tuples from the city relation.  You can do the same for country if you want.

In [6]:
from reframe import Relation
city = Relation('city.csv',sep='|')
country = Relation('country.csv',sep='|')
city.head()

,id,name,countrycode,district,population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


All relational operators have a similar form:

originalrelation.operator(parameters)

The dot is important as it tells the operator which relation it should operate on.  The additional parameters inform the operator how it should behave.

## Project

The project operator tranforms a relation into another by only keeping the list of columns specified as a parameter to the operator.  In that light you should always visualize the project operator as narrowing the original as shown in the picture below.

![](project_shape.png)

Lets look at a simple example using the city relation.  Lets project the name and district from the original city relation.  Notice that the names and the districts nicely match up with the names from the original, but that we have eliminated  the id, countrycode and population columns.

In [8]:
city.project(['name','district']).head()

,name,district
0,Kabul,Kabol
1,Qandahar,Qandahar
2,Herat,Herat
3,Mazar-e-Sharif,Balkh
4,Amsterdam,Noord-Holland


Now lets project the countrycode column.  You might expect to see the five rows tuples with the values AFG, AFG, AFG, AFG, NLD.   But remember that relations do not have duplicate rows, so in fact you will not see four tuples containing AFg.  Lets look:

In [10]:
city.project(['countrycode']).head()

,countrycode
0,AFG
4,NLD
32,ANT
33,ALB
34,DZA


## Query

Our next relational operator is the query operator.  The query operator allows us to create a new relation that retains a subset of the original tuples.  The tuples that are retained depend on the condition we provide to the query operator.  Typically a query will compare one or more values in the tuple against a constant value, or against each other.  The query operator keeps all of the columns intact, it only elimiates rows!  So in terms of changing the shape of a relation we can see that query looks like this:

![](query_shape.png)

The kinds of comparisons we can make include:

* ``==``
* ``!=``
* ``>=``
* ``>``
* ``<``
* ``<=``

The example below shows a query corresponding to the questions:  List all of the data about all of the cities that have a population more than 1 million.

In [12]:
city.query("population > 1000000").head()

,id,name,countrycode,district,population
0,1,Kabul,AFG,Kabol,1780000
34,35,Alger,DZA,Alger,2168000
55,56,Luanda,AGO,Luanda,2022000
68,69,Buenos Aires,ARG,Distrito Federal,2982146
69,70,La Matanza,ARG,Buenos Aires,1266461


As mentioned we can compare one column with another.  For example lets try a query that retains all of the tuples where the name of the city is the same as the name of the district.

In [14]:
city.query("name == district").head()

,id,name,countrycode,district,population
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
7,8,Utrecht,NLD,Utrecht,234323
10,11,Groningen,NLD,Groningen,172701
33,34,Tirana,ALB,Tirana,270000


For one final example lets write a query that retains all of the rows where the countrycode is ``'AFG'``.  This is a bit tricky as we call ``'AFG'`` a string literal, and it must be enclosed by quotes.  Names of columns never have to be surrounded by quotes, but the value of a column within a tuple that not numeric, must always be enclosed in quotes.

You will also notice that in the example below we do not use the ``head()`` at the end of our query, this is becuase the result is short enough that we don't need to use head to restrict the number of rows that we show.  

In [16]:
city.query("countrycode == 'AFG'")

,id,name,countrycode,district,population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800


## Sort

The sort relational is even easier than either of the previous two in that all it does is reorder the data that is already there.  The parameters to the sort operator are a list of columns.  The first column on the list is the primary sort key, which means we sort the tuples in the relation from smallest to largest according the values in that column.  If another column is given, then we use the second column as a tie breaker when the values in the first column are equal.

![](sort.png)

Here is a simple example where we sort the cities in order by population.  If we want to see the cities sorted from biggest to smallest we can supply an additional parameter ``ascending=False``

In [17]:
city.sort(['population']).head()

,id,name,countrycode,district,population
2911,2912,Adamstown,PCN,,42
2316,2317,West Island,CCK,West Island,167
3332,3333,Fakaofo,TKL,Fakaofo,300
3537,3538,Città del Vaticano,VAT,,455
2315,2316,Bantam,CCK,Home Island,503


In [20]:
city.sort(['population'],ascending=False).head()

,id,name,countrycode,district,population
1023,1024,Mumbai (Bombay),IND,Maharashtra,10500000
2330,2331,Seoul,KOR,Seoul,9981619
205,206,São Paulo,BRA,São Paulo,9968485
1889,1890,Shanghai,CHN,Shanghai,9696300
938,939,Jakarta,IDN,Jakarta Raya,9604900


Next lets look at an example of sorting using two columns.  Our first sort key will be the country code so that we can gather together all of the cities in the same country.  But within each country we will sort the cities from smallest to largest by population.

In [19]:
city.sort(['countrycode','population']).head(10)

,id,name,countrycode,district,population
128,129,Oranjestad,ABW,,29034
3,4,Mazar-e-Sharif,AFG,Balkh,127800
2,3,Herat,AFG,Herat,186800
1,2,Qandahar,AFG,Qandahar,237500
0,1,Kabul,AFG,Kabol,1780000
59,60,Namibe,AGO,Namibe,118200
58,59,Benguela,AGO,Benguela,128300
57,58,Lobito,AGO,Benguela,130000
56,57,Huambo,AGO,Huambo,163100
55,56,Luanda,AGO,Luanda,2022000


## Groupby

The next operator is groupby, which gives us a huge amount of additional power.  Think of the groupby operator as a kind of extension of sort.  That is group by first sorts the relation according to the columns given it.  Think of a group as the set of columns that all have the same value for the column we are grouping on.  

![](groupby1.png)


But then the real power of the groupby operator is that we can summarize the data in the group in one of several ways.  

* count -- simply count the number of rows in the group.
* mean -- take the average of a numerical value in a column for all the rows in the group
* max -- find the max of a numerical value in a column for all the rows in the group
* min -- find the min of a numerical value in a column for all the rows in the group
* sum -- take the average of a numerical value in a column for all the rows in the group
* median -- find the median a numerical value in a column for all the rows in the group

![](groupby2.png)


The image you should have in mind is that after we have grouped all the rows together than have the same value for the groupby column, we squash them all together into a single row, with some summary (or **aggregate** value.)

Using grouby transforms the original relation in two ways:

* It reduces the number of rows since we are grouping rows together and then squashing them down to a single row.  
* It also reduces the number of columns to include **only** the groupby column and the result of our aggregate operator.

For example, lets use groupby to count the number of cities in each country.  To do this we will group by the countrycode and count the number of names

In [21]:
city.groupby(['countrycode']).count('name').head()

,countrycode,count_name
0,ABW,1
1,AFG,4
2,AGO,5
3,AIA,2
4,ALB,1


Lets also look at an example where we group by two columns.  In this example we will first group by country, and then within each country we will group by district.  This will tell us the total population of the district within each country.

In [25]:
city.groupby(['countrycode','district']).sum('population').head(10)

,countrycode,district,sum_population
0,ABW,,29034
1,AFG,Balkh,127800
2,AFG,Herat,186800
3,AFG,Kabol,1780000
4,AFG,Qandahar,237500
5,AGO,Benguela,258300
6,AGO,Huambo,163100
7,AGO,Luanda,2022000
8,AGO,Namibe,118200
9,AIA,,1556


## Rename

The rename operator allows us to change the name of a column.  It has no other effect on the relation besides giving the column a new name.


In [26]:
city.rename('name','cityname').head()

,id,cityname,countrycode,district,population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


## Extend

The extend operator allows us to ADD a new column to the relation.  For example lets say we wanted to add a column to the country relation that shows us the population density.  We could divide the land area of the country by its population giving us the population density

In [31]:
country.extend('density',country.population/country.surfacearea).head()



,code,name,continent,region,surfacearea,indepyear,population,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2,density
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090,1919,22720000,45.9,5976,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1,AF,34.841816
1,NLD,Netherlands,Europe,Western Europe,41526,1581,15864000,78.3,371362,360478,Nederland,Constitutional Monarchy,Beatrix,5,NL,382.025719
2,ANT,Netherlands Antilles,North America,Caribbean,800,NaN,217000,74.7,1941,NaN,Nederlandse Antillen,Nonmetropolitan Territory of The Netherlands,Beatrix,33,AN,271.250000
3,ALB,Albania,Europe,Southern Europe,28748,1912,3401200,71.6,3205,2500,Shqipëria,Republic,Rexhep Mejdani,34,AL,118.310839
4,DZA,Algeria,Africa,Northern Africa,2381740,1962,31471000,69.7,49982,46966,Al-Jazair/Algérie,Republic,Abdelaziz Bouteflika,35,DZ,13.213449


OK, thats a lot of information, so lets combine extend and project to show us the name of the country, the surface area, the population, and the calculated density

In [32]:
country.extend('density',country.population/country.surfacearea).project(['name','surfacearea','population','density']).head()

,name,surfacearea,population,density
0,Afghanistan,652090,22720000,34.841816
1,Netherlands,41526,15864000,382.025719
2,Netherlands Antilles,800,217000,271.250000
3,Albania,28748,3401200,118.310839
4,Algeria,2381740,31471000,13.213449
